In [24]:
#| default_exp create_thumbnails

# Normalize images and create thumbnails

In [25]:
#| export

import numpy as np
import daft
from PIL import Image

## 1. Normalize

In [26]:
# | export

def _autocontrast_normed(img_arr: np.array) -> np.array:
    """autocontrast from keras - added normalization"""
    x = img_arr.astype(float)
    mean_before = x.mean()
    ## autocontrast from Keras
    x = x - np.min(x)
    x_max = np.max(x)
    if x_max != 0:  x /= x_max
    x *= 255

    ## return to average lightness of input image
    mean_shift = x.mean() - mean_before

    print(f"Shape after: {x.shape}")
    return np.clip(x - mean_shift, 0, 255).astype("uint8")

In [27]:
# | export

@daft.udf(return_dtype=daft.DataType.image(mode="RGB"))
def _autocontrast_img(arrs: daft.Series) -> np.array:
    """apply autocontrast to an image column"""
    return [_autocontrast_normed(img) for img in arrs.to_pylist()]

In [28]:
#| export

def create_autocontrast_col(df: daft.DataFrame, image_col: str="img") -> daft.DataFrame:
    """create autocontrast image column"""
    return df.with_column("img_autocontrast", _autocontrast_img(df["img"]))

## 2. Create thumbnails

In [29]:
#| hide

import nbdev; nbdev.nbdev_export()